In [1]:
import os

In [ ]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/vbs.codestuff/Project_01.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="vbs.codestuff"
os.environ["MLFLOW_TRACKING_PASSWORD"]="b7378850fb9748992f5d0f157e192c883bc11a78"

In [5]:
import os
%pwd

'c:\\Users\\Vaibhav\\Desktop\\Course\\12_project_1\\Project_01\\research'

In [6]:
os.chdir("../")
%pwd

'c:\\Users\\Vaibhav\\Desktop\\Course\\12_project_1\\Project_01'

In [22]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [8]:
from src.datascience_project.constants import *
from src.datascience_project.utils.common import read_yaml, create_directory, save_json

In [30]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directory([self.config.artifacts_root])

    def get_model_evaluation_config(self):
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directory([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            test_data_path = config.test_data_path, 
            model_path = config.model_path,
            all_params = params,
            metric_file_name = config.mertics_filename,
            target_column = schema.name,
            mlflow_uri = "https://dagshub.com/vbs.codestuff/Project_01.mlflow"
        )

        return model_evaluation_config


In [16]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [31]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        rsme = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rsme, mae, r2
    
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_X = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[self.config.target_column]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        trakcing_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted_quantities = model.predict(test_X)

            (rsme, mae, r2) = self.eval_metrics(test_y, predicted_quantities)

            # Saving mertrics at local
            scores = {"rsme": rsme, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rsme", rsme)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)

            if trakcing_url_type_store != 'file':
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticNetModel")
            else:
                mlflow.sklearn.log_model(model, "model")

In [32]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2025-05-04 17:40:14,165: INFO: common: yaml file config\config.yaml loaded successfully]
[2025-05-04 17:40:14,169: INFO: common: yaml file params.yaml loaded successfully]
[2025-05-04 17:40:14,174: INFO: common: yaml file schema.yaml loaded successfully]
[2025-05-04 17:40:14,177: INFO: common: created directory at: artifacts]
[2025-05-04 17:40:14,179: INFO: common: created directory at: artifacts/model_evaluation]
[2025-05-04 17:40:14,587: INFO: common: json file saved as: artifacts\model_evaluation\metrics.json]


2025/05/04 17:40:24 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'ElasticNetModel'.
2025/05/04 17:40:29 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticNetModel, version 1
Created version '1' of model 'ElasticNetModel'.


🏃 View run spiffy-grouse-820 at: https://dagshub.com/vbs.codestuff/Project_01.mlflow/#/experiments/0/runs/948d12a465ce4aba93ea13ee989cb6ed
🧪 View experiment at: https://dagshub.com/vbs.codestuff/Project_01.mlflow/#/experiments/0
